In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import re

# Rank Swapping

In [2]:
numeric_cols = ['bin', 'amount']
all_cols = ['txid', 'bookingdate', 'issuercountrycode', 'txvariantcode', 'bin',
       'amount', 'currencycode', 'shoppercountrycode', 'shopperinteraction',
       'simple_journal', 'cardverificationcodesupplied', 'cvcresponsecode',
       'creationdate', 'accountcode', 'mail_id', 'ip_id', 'card_id']

In [4]:
from rank_swapping import rank_swapping
from models import get_DecisionTree_roc
from sklearn.model_selection import train_test_split
from preprocessing import preprocess

fig = plt.figure(figsize=(10,8))
plt.xlabel("FPR")
plt.ylabel("TPR")

for p in [0, 1, 2, 5, 10]:
    data = "./data/data_for_student_case.csv"
    df1 = pd.read_csv(data)

    #rankswap
    swap_cols = numeric_cols
    df1 = rank_swapping(df1, p, swap_cols)

    #preprocess
    df1 = preprocess(df1)

    columns = ['txid', 'bookingdate', 'issuercountrycode', 'txvariantcode', 'bin',
       'amount', 'currencycode', 'shoppercountrycode', 'shopperinteraction',
       'simple_journal', 'cardverificationcodesupplied', 'cvcresponsecode',
       'creationdate', 'accountcode', 'mail_id', 'ip_id', 'card_id'],
    X = df1[columns]
    Y = df1['simple_journal'].to_numpy(dtype=np.int)

    #build and evaluate model
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    fpr_model, tpr_model = get_DecisionTree_roc(X_train, X_test, Y_train, Y_test)
    plt.plot(fpr_model, tpr_model, label="p = "+str(p))

plt.legend()
plt.show()

ValueError: could not convert string to float: '2015-11-09 14:26:51'